In [1]:
import pandas as pd
import schedule
import time
import json
import requests
from pandas import json_normalize

In [2]:
coin_list = ['bitcoin', 'ethereum', 'tether', 'ripple', 'litecoin', 'cardano', 'dogecoin', 'matic-network', 'usd-coin', 'binancecoin', 'binance-usd']
exchange = ['binance', 'coinbase', 'okx', 'huobi']

#timer function to call function create_exchange_df() every 5 minutes
# def api_timer(exchange, coin_list):
#     schedule.every(2).minutes.do(create_exchange_df(exchange, coin_list))
    
#     while True:
#         schedule.run_pending()
#         time.sleep(3)

#function loops through exchange list and calls get_live_data for each exchange to get coin data        
def create_exchange_df(exchange, coin_list):
    for x in exchange:
        exchange_data = get_live_data(x, coin_list)
        exchange_df = pd.DataFrame(exchange_data)
    return exchange_df
 
    
#function loops through coin list to call api data from coingecko, clean dat and return a dataframe all_coin_info_df
def get_live_data (exchange, coin_list):
    exchange_df = pd.DataFrame()
    try:
        for c in coin_list:
            url = (f"https://api.coingecko.com/api/v3/exchanges/{exchange}/tickers?coin_ids={c}")
            response = requests.get(url).json()
            df = json_normalize(response['tickers'][0])
            exchange_df = exchange_df.append(df, ignore_index=True) 
    
    
    except json.JSONDecodeError:
        pass
    finally:
        #cleans dataframe data
        exchange_df = exchange_df[['base',
          'target', 'last', 'volume', 'bid_ask_spread_percentage',
          'timestamp', 'market.name', 'market.identifier', 'market.has_trading_incentive',
          'converted_last.usd', 'converted_volume.usd']]
        exchange_df["timestamp"] = pd.to_datetime(exchange_df["timestamp"], infer_datetime_format =True, utc = True)
        exchange_df = exchange_df.reset_index()
        exchange_df = exchange_df.set_index('timestamp')
        print (exchange_df)
        return exchange_df
    
create_exchange_df(exchange, coin_list)   

                           index  base target        last        volume  \
timestamp                                                                 
2022-12-18 04:46:49+00:00      0   BTC   USDT  16767.2800  1.221947e+05   
2022-12-18 04:36:10+00:00      1   ETH   USDT   1193.8100  2.694027e+05   
2022-12-18 04:40:04+00:00      2  USDT    TRY     19.0700  3.949661e+07   
2022-12-18 04:44:50+00:00      3   XRP   USDT      0.3538  1.666381e+08   
2022-12-18 04:46:57+00:00      4   LTC   USDT     65.4500  1.121475e+06   

                           bid_ask_spread_percentage market.name  \
timestamp                                                          
2022-12-18 04:46:49+00:00                   0.010060     Binance   
2022-12-18 04:36:10+00:00                   0.010838     Binance   
2022-12-18 04:40:04+00:00                   0.052411     Binance   
2022-12-18 04:44:50+00:00                   0.028249     Binance   
2022-12-18 04:46:57+00:00                   0.015281     Binance  

KeyError: "None of [Index(['base', 'target', 'last', 'volume', 'bid_ask_spread_percentage',\n       'timestamp', 'market.name', 'market.identifier',\n       'market.has_trading_incentive', 'converted_last.usd',\n       'converted_volume.usd'],\n      dtype='object')] are in the [columns]"